In [6]:
import numpy as np
from sklearn.datasets import fetch_california_housing
import tensorflow as tf
from pprint import pprint

# 数据准备

In [7]:
housing = fetch_california_housing()
pprint(housing)

{'DESCR': '.. _california_housing_dataset:\n'
          '\n'
          'California Housing dataset\n'
          '--------------------------\n'
          '\n'
          '**Data Set Characteristics:**\n'
          '\n'
          '    :Number of Instances: 20640\n'
          '\n'
          '    :Number of Attributes: 8 numeric, predictive attributes and the '
          'target\n'
          '\n'
          '    :Attribute Information:\n'
          '        - MedInc        median income in block\n'
          '        - HouseAge      median house age in block\n'
          '        - AveRooms      average number of rooms\n'
          '        - AveBedrms     average number of bedrooms\n'
          '        - Population    block population\n'
          '        - AveOccup      average house occupancy\n'
          '        - Latitude      house block latitude\n'
          '        - Longitude     house block longitude\n'
          '\n'
          '    :Missing Attribute Values: None\n'
          

In [17]:
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

print(housing_data_plus_bias.shape)
print(scaled_housing_data_plus_bias.shape)

(20640, 9)
(20640, 9)


# 创建`线性回归模型的正规方程`计算图

In [10]:
# 创建计算图
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
X_trans = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_trans, X)), X_trans), y)

# 会话执行计算图
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[-3.7171074e+01]
 [ 4.3633682e-01]
 [ 9.3871783e-03]
 [-1.0717344e-01]
 [ 6.4540231e-01]
 [-4.1238391e-06]
 [-3.7809242e-03]
 [-4.2373490e-01]
 [-4.3720812e-01]]


# 创建`批量梯度下降算法`的计算图

In [ ]:
# ===================================================
# 配置项
# ===================================================
n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

# ===================================================
# 创建计算图
# ===================================================

def train_op():
    # 
    # X = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X")
    # y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")

    # 构造占位符节点
    X = tf.placeholder(tf.float32, shape = (None, n + 1), name = "X")
    y = tf.placeholder(tf.float32, shape = (None, 1), name = "y")

    theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name = "theta")
    y_pred = tf.matmul(X, theta, name = "predictions")
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name = "mse")

    # -------------------
    # 手动计算梯度
    # -------------------
    # gradients = 2 / m * tf.matmul(tf.transpose(X), error)
    # training_op = tf.assign(theta, theta - learning_rate * gradients)

    # -------------------
    # 使用自动微分
    # -------------------
    # gradients = tf.gradients(mse, [theta])[0] 
    # training_op = tf.assign(theta, theta - learning_rate * gradients)

    # -------------------
    # 使用优化器
    # -------------------
    # 梯度下降优化器
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
    # training_op = optimizer.minimize(mse)

    # -------------------
    # 动量优化器
    # -------------------
    optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)
    training_op = optimizer.minimize(mse)
    
    return train_op

def fetch_batch(epoch, batch_index, batch_size):
    [...]
    return X_batch, y_batch


# ===================================================
# 会话执行计算图
# ===================================================
init = tf.global_variabels_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict = {X: X_batch, y: y_batch})
    best_theta = theta.eval()

# ===================================================
# 保存模型
# ===================================================
init = tf.global_variabels_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

# ===================================================
# 加载已保存的模型
# ===================================================
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    
